In [1]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import torch
import pandas as pd
from datasets import Dataset
import numpy as np
from transformers import pipeline
from transformers_interpret import MultiLabelClassificationExplainer

# Interpreting the model trained on fake data and tested on real data

In [8]:
dataset_filename = "../linguistic_model/data/answers.csv"
# answers from the CLAN dataset PWA + control 

# dataset_filename = "../linguistic_model/data/synthetic_clan_c4_2.csv"
data = pd.read_csv(dataset_filename, encoding='utf8', index_col=False).drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1).dropna()

In [26]:
data

,scenario,source_file,original_text,preprocessed_text,label,predicted
0,Important_Event,scale01a.cha,&=points:self maybe pretty good . [+ gram] 32...,maybe pretty good,1,0
1,Window,1030-3.cha,he was looking out &+wu window . 53709_58459,he was looking out window,1,0
2,Stroke,MSU05b.cha,and speech was very good &-uh I [//] you_know ...,and speech was very good i you know,1,0
3,Cinderella,kurland29c.cha,xxx dance and dance xxx . [+ jar] 1056770_105...,dance and dance,1,0
4,Stroke,scale31a.cha,<my eyes> [//] &=points:eye my eye . [+ gram] ...,my eyes my eye,1,0
...,...,...,...,...,...,...
1995,Cat,MSUC09a.cha,well ‡ a cat is stuck up in the tree . 456603...,well a cat is stuck up in the tree,0,0
1996,Speech,fridriksson06b.cha,+< &-uh &+s &=ges:so_so yeah . 19964_20534,yeah,1,0
1997,Sandwich,7-2.cha,but you put those things together . 1316789_1...,but you put those things together,0,1
1998,Flood,tap11a.cha,&=points:picture_girl yes &=points:picture_man...,yes yes,1,0


In [27]:
data_fn = data[(data["label"]==1) & (data["predicted"]==0)]
data_fp = data[(data["label"]==0) & (data["predicted"]==1)]
data_tp = data[(data["label"]==0) & (data["predicted"]==0)]
data_tn = data[(data["label"]==1) & (data["predicted"]==1)]

In [28]:
model_name = "models/bert_reverse"
tokenizer_name = "models/bert_reverse"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, return_tensor="pt")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe = pipeline("text-classification", model=model_name, tokenizer=tokenizer_name, device=device)
cls_explainer = MultiLabelClassificationExplainer(model, tokenizer)

In [23]:
model.config.id2label

{0: 'CONTROL', 1: 'BROCA'}

In [7]:
dataset = Dataset.from_pandas(data_tn)
def preprocess_function(examples):
    return tokenizer(examples["preprocessed_text"], padding=True, truncation=True, return_tensors="pt", max_length=512)
    # return tokenizer(examples["modified"], padding=True, truncation=True, return_tensors="pt")

tokenized_data = dataset.map(preprocess_function, batched=True).with_format("torch")
# tokenized_data_split = tokenized_data.train_test_split(test_size=0.2, seed=42)
num = 0
n = 5
for text, label in zip(list(tokenized_data["preprocessed_text"]), np.array(tokenized_data["label"])): 
    if num != n:
        print("True: ", model.config.id2label[label], "Pred: ", pipe(text)[0]["label"])
        print("Text: ", text)
        cls_explainer(text)
        cls_explainer.visualize()
        print("----------------------------------------------------------------------")
        num += 1

Map:   0%|          | 0/88 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


True:  BROCA Pred:  BROCA
Text:  i i forgot this one in there.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.53),CONTROL,-0.70,[CLS] i i forgot this one in there . [SEP]
n/a,(0.35),BROCA,0.39,[CLS] i i forgot this one in there . [SEP]


----------------------------------------------------------------------
True:  BROCA Pred:  BROCA
Text:  okay you.


...

----------------------------------------------------------------------
True:  BROCA Pred:  BROCA
Text:  my this.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.25),CONTROL,-1.68,[CLS] my this . [SEP]
n/a,(0.73),BROCA,1.69,[CLS] my this . [SEP]


----------------------------------------------------------------------
True:  BROCA Pred:  BROCA
Text:  i good.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.31),CONTROL,-1.19,[CLS] i good . [SEP]
n/a,(0.64),BROCA,1.20,[CLS] i good . [SEP]


----------------------------------------------------------------------


In [29]:
dataset = Dataset.from_pandas(data_fn)
def preprocess_function(examples):
    return tokenizer(examples["preprocessed_text"], padding=True, truncation=True, return_tensors="pt", max_length=512)
    # return tokenizer(examples["modified"], padding=True, truncation=True, return_tensors="pt")

tokenized_data = dataset.map(preprocess_function, batched=True).with_format("torch")
# tokenized_data_split = tokenized_data.train_test_split(test_size=0.2, seed=42)
num = 0
n = 100
for text, label in zip(list(tokenized_data["preprocessed_text"]), np.array(tokenized_data["label"])): 
    if num != n:
        print("True: ", model.config.id2label[label], "Pred: ", pipe(text)[0]["label"])
        print("Text: ", text)
        cls_explainer(text)
        cls_explainer.visualize()
        print("----------------------------------------------------------------------")
        num += 1

Map:   0%|          | 0/902 [00:00<?, ? examples/s]

True:  BROCA Pred:  CONTROL
Text:  maybe pretty good


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.81),CONTROL,1.70,[CLS] maybe pretty good [SEP]
n/a,(0.19),BROCA,-1.68,[CLS] maybe pretty good [SEP]


----------------------------------------------------------------------
True:  BROCA Pred:  CONTROL
Text:  he was looking out window


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.83),CONTROL,1.24,[CLS] he was looking out window [SEP]
n/a,(0.18),BROCA,-0.99,[CLS] he was looking out window [SEP]


...

----------------------------------------------------------------------
True:  BROCA Pred:  CONTROL
Text:  but here inside to here


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.74),CONTROL,0.67,[CLS] but here inside to here [SEP]
n/a,(0.27),BROCA,-0.34,[CLS] but here inside to here [SEP]


----------------------------------------------------------------------
True:  BROCA Pred:  CONTROL
Text:  no


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.51),CONTROL,-1.00,[CLS] no [SEP]
n/a,(0.52),BROCA,1.00,[CLS] no [SEP]


----------------------------------------------------------------------


In [17]:
tokenized_data

Dataset({
    features: ['Unnamed: 0', 'Unnamed: 0.1', 'scenario', 'source_file', 'original_text', 'preprocessed_text', 'label', 'predicted', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 912
})

# Interpreting the model trained on real data and tested on fake data

In [7]:
# dataset_filename = "../preprocessing/data/processed_merge.csv"
dataset_filename = "../linguistic_model/data/synthetic_clan_merge_imdb_test.csv"
data = pd.read_csv(dataset_filename, encoding='utf8', index_col=False).sample(frac=1)

In [3]:
model_name = "models/bert_finetuned"
tokenizer_name = "models/bert_finetuned"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, return_tensor="pt")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe = pipeline("text-classification", model=model_name, tokenizer=tokenizer_name, device=device)
cls_explainer = MultiLabelClassificationExplainer(model, tokenizer)

In [4]:
model.config.id2label

{0: 'CONTROL', 1: 'BROCA'}

In [8]:
dataset = Dataset.from_pandas(data)
def preprocess_function(examples):
    return tokenizer(examples["modified"], padding=True, truncation=True, return_tensors="pt")
tokenized_data = dataset.map(preprocess_function, batched=True).with_format("torch")
tokenized_data_split = tokenized_data.train_test_split(test_size=0.2, seed=42)

Map:   0%|          | 0/3456 [00:00<?, ? examples/s]

### Observations (model trained on real data, tested on fake)

* Model looks at ' again for same point as mentioned above

In [9]:
num = 0
n = 10
for text, label in zip(list(tokenized_data_split["test"]["modified"]), np.array(tokenized_data_split["test"]["label"])): 
    if num != n:
        print("True: ", model.config.id2label[label], "Pred: ", pipe(text)[0]["label"])
        print("Text: ", text)
        cls_explainer(text)
        cls_explainer.visualize()
        print("----------------------------------------------------------------------")
        num += 1

C:\Users\imruh\miniconda3\envs\thesis\lib\site-packages\transformers\pipelines\base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


True:  CONTROL Pred:  CONTROL
Text:  now no you could not see exactly what was going to happen but by the time the twist actually occurred i for one did not even care.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  they should have given the kid a wide berth sang it a lullaby then ran the train into a ravine.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.97),CONTROL,3.41,[CLS] they should have given the kid a wide berth sang it a lullaby then ran the train into a ravine . [SEP]
n/a,(0.02),BROCA,-3.32,[CLS] they should have given the kid a wide berth sang it a lullaby then ran the train into a ravine . [SEP]


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  the choices of the main character ty hackett are quite uninspired and the final with the reward is quite childish.


...

----------------------------------------------------------------------
True:  BROCA Pred:  CONTROL
Text:  i i to their their eye when i realised it be made two thousand and one and not one thousand nine hundred and seventy one.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  back in one thousand nine hundred and seventy that scene might have worked on a certain level but today that moment reeks of desperation.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  this could have been a good movie that shows the struggles of dylan thomas during wwii and how strong the sentiment was against men who somehow managed to avoid serving in the military then.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  vd gives a particularly uninspired performance and given he is never been a robert de niro sort of actor that cannot be good.


----------------------------------------------------------------------
